# Usage of the Mitigator object

This notebook shows the usage of the Mitigator object.
 
**Important: We assume that the user is able to evaluate the expectation value of an input circuit and also to increase the noise level of the backend.**

In [1]:
import numpy as np

# Error mitigation package
from mitiq import run_mitigation, fun_mitigator

In [2]:
# IBM SDK
import qiskit
from qiskit import QuantumCircuit

# Noise simulation packages
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error

# Other tools


The following function generates a random sequence of Pauli gates which is equivalent to the identity.

In [3]:
def random_identity_circuit(depth=None):
    """Returns a single-qubit identity circuit based on Pauli gates."""
    
    # initialize a quantum circuit with 1 qubit and 1 classical bit
    circuit = QuantumCircuit(1, 1)
    
    # index of the (inverting) final gate: 0=I, 1=X, 2=Y, 3=Z
    k_inv = 0

    # apply a random sequence of Pauli gates
    for _ in range(depth):
        # random index for the next gate: 1=X, 2=Y, 3=Z
        k = np.random.choice([1, 2, 3])
        # apply the Pauli gate "k"
        if k == 1:
            circuit.x(0)
        elif k == 2:
            circuit.y(0)
        elif k == 3:
            circuit.z(0)
        
        # update the inverse index according to 
        # the product rules of Pauli matrices k and k_inv
        if k_inv == 0:
            k_inv = k
        elif k_inv == k:
            k_inv = 0
        else:
            _ = [1, 2, 3]
            _.remove(k_inv)
            _.remove(k)
            k_inv = _[0]
    
    # apply the final inverse gate
    if k_inv == 1:
        circuit.x(0)
    elif k_inv == 2:
        circuit.y(0)
    elif k_inv == 3:
        circuit.z(0)
        
    return circuit

We assume that the user is able to define a function which:
* takes as arguments a circuit and a noise stretch factor,
* executes the circuit in an arbitrary backend,
* returns the final expectation value.

The user can directly decorate its own function with a single line of code `@mitigate()`, placed right above the function definition. For example:

In [4]:
def noisy_run(circuit=None, stretch=1):
    """Execute a circuit on a noisy device and returns the expectation value of 
    the final measurement."""
    
    shots = 10 ** 5       # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch
    
    # initialize a qiskit noise model
    noise_model = NoiseModel()
    
    # we assume a depolarizing error for each gate of the standard IBM basis set (u1, u2, u3)
    noise_model.add_all_qubit_quantum_error(depolarizing_error(noise, 1), ['u1', 'u2', 'u3'])
    
    # execution of the experiment
    job = qiskit.execute(circuit, 
                         backend=backend, 
                         basis_gates=['u1', 'u2', 'u3'],
                         # we want all gates to be actually applied,
                         # so we skip any circuit optimization
                         optimization_level=0, 
                         noise_model=noise_model,
                         shots=shots)
    results = job.result()
    counts = results.get_counts()
    expval = counts['0'] / shots
    return expval

Let us use the function without error mitigation.

In [5]:
# beckend initialization
backend = qiskit.Aer.get_backend('qasm_simulator')
rand_circ = random_identity_circuit(depth=100)
rand_circ.measure(0, 0)

# execution with automatic error mitigation
noisy_run(rand_circ, stretch=1)

0.74593

Now we use the error the mitigation toolkit (without using the decorator):

In [6]:
# beckend initialization
backend = qiskit.Aer.get_backend('qasm_simulator')
rand_circ = random_identity_circuit(depth=100)
rand_circ.measure(0, 0)

# execution with automatic error mitigation
run_mitigation(noisy_run, rand_circ, order=2)

0.9351400000000001

The user can also directly decorate its own function with a single line of code `@fun_mitigator()`, placed right above the function definition. For example:

In [7]:
@fun_mitigator(order=2)
def magic_run(circuit=None, stretch=1):
    """Execute a circuit on a noisy device and returns the expectation value of 
    the final measurement."""
    
    shots = 10 ** 5       # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch
    
    # initialize a qiskit noise model
    noise_model = NoiseModel()
    
    # we assume a depolarizing error for each gate of the standard IBM basis set (u1, u2, u3)
    noise_model.add_all_qubit_quantum_error(depolarizing_error(noise, 1), ['u1', 'u2', 'u3'])
    
    # execution of the experiment
    job = qiskit.execute(circuit, 
                         backend=backend, 
                         basis_gates=['u1', 'u2', 'u3'],
                         # we want all gates to be actually applied,
                         # so we skip any circuit optimization
                         optimization_level=0, 
                         noise_model=noise_model,
                         shots=shots)
    results = job.result()
    counts = results.get_counts()
    expval = counts['0'] / shots
    return expval

Now the user can directly apply the function `magic_run` to evaluate circuits. Error mitigation is automatically applied behind the scenes!

In [8]:
# beckend initialization
backend = qiskit.Aer.get_backend('qasm_simulator')
rand_circ = random_identity_circuit(depth=100)
rand_circ.measure(0, 0)

# execution with automatic error mitigation
magic_run(rand_circ)

0.9362800000000002